In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
site_url = 'https://profiles.stanford.edu/browse/school-of-engineering/programs-centers-and-institutes/institute-for-computational-and-mathematical-engineering-icme?p=1&affiliations=capFaculty&ps=100'

In [3]:
response = requests.get(site_url)
response.status_code

In [4]:
html = response.content

In [5]:
soup = BeautifulSoup(html, 'lxml')

In [6]:
divs = soup.find_all("div", {"class": "mini-profile"})

In [7]:
divs[0].find("h4").string

In [8]:
faculty_names = [div.find("h4").string for div in divs]
faculty_names[:5]

In [9]:
divs[0].find('h5').text

In [10]:
faculty_association = [div.find('h5').text for div in divs]
faculty_association[:5]

In [11]:
divs[0].find('a')['href']

In [12]:
profile_extensions = [div.find('a')['href'] for div in divs]
profile_extensions[:5]

In [14]:
base_url = 'https://profiles.stanford.edu/'

In [16]:
personal_website = [base_url + profile_extension for profile_extension in profile_extensions]
personal_website[:5]

In [18]:
#orcid has a public api available
orcid_website = []
for site in personal_website:
    url = site
    response = requests.get(url)
#     if response.ok is False:
#         raise ValueError ("Url didn't work")
    html = response.content
    soup = BeautifulSoup(html, 'lxml')
    if soup.find('a', {'target': 'orcid.widget'}):
        orcid_website.append(soup.find('a', {'target': 'orcid.widget'})['href'])
    else:
        orcid_website.append('None')

In [157]:
research_interests = []
published_articles = []

for url in personal_website:
    print(i)
    response = requests.get(url)
    response.status_code
    html = response.content
    soup = BeautifulSoup(html, 'lxml')
    #Faculty Research Interests, Teaching, Professional Activities
    if soup.find('div', {'id': 'currentResearchAndScholarlyInterestsContent'}):
        research_interests.append(soup.find('div', {'id': 'currentResearchAndScholarlyInterestsContent'}).find('p').text.strip('Research'))
    else:
        research_interests.append('Not Provided')
    
    #Publication details
    publications = soup.find_all(class_ = 'publication article')
    publication_details = {'pub_title' : [], 'pub_authors' : [], 'pub_year' : []}

    for publication in publications:
        if publication.find('span', class_ = 'title'):
            publication_details['pub_title'].append(publication.find('span', class_ = 'title').text.strip())
        if publication.find('span', {'class':'authors'}):
            publication_details['pub_authors'].append(publication.find('span', {'class':'authors'}).text.strip())
        if publication.find('span', {'class':'year'}):
            publication_details['pub_year'].append(publication.find('span', {'class':'year'}).text)
    
    published_articles.append(publication_details)

In [20]:
faculty_df = pd.DataFrame()

In [160]:
faculty_df['faculty_name'] = faculty_names
faculty_df['faculty_association'] = faculty_association
faculty_df['profile'] = personal_website
faculty_df['orcid_website'] = orcid_website
faculty_df['research_interests'] = research_interests
faculty_df['published_articles'] = published_articles

In [162]:
faculty_df

In [165]:
faculty_df.to_csv('stanford_faculty_df.csv')